## 第8章　CAEを使ったノイズ除去

**共通モジュールの読み込み**

In [1]:
import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


**リスト8.1:MNISTデータセットの読み込みと前処理**

In [5]:
from tensorflow.python.keras.datasets import mnist


(x_train, _), (x_test, _) = mnist.load_data() 
# CNNで扱いやすい形に変換
x_train = x_train.reshape(-1, 28, 28, 1)  # (60000, 28, 28) -> (60000, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1) 
# 画像を0-1の範囲に正規化
x_train = x_train/255. 
x_test = x_test/255. 

In [6]:
array_to_img(x_train[0])

**リスト8.2:マスキングノイズを加えて、擬似ノイズデータを生成**

In [7]:
def make_masking_noise_data(data_x, percent=0.1):
    size = data_x.shape
    masking = np.random.binomial(n=1, p=percent, size=size) # 二項分布（互いに独立したベルヌーイ試行を n 回行ったときにある事象が何回起こるかの確率分布）によるノイズ
    return data_x*masking

x_train_masked = make_masking_noise_data(x_train)
x_test_masked = make_masking_noise_data(x_test)

**リスト8.3:ガウシアンノイズを加えて、擬似ノイズデータを生成**

In [8]:
def make_gaussian_noise_data(data_x, scale=0.8):
    gaussian_data_x = data_x + np.random.normal(loc=0, scale=scale, size=data_x.shape)
    gaussian_data_x = np.clip(gaussian_data_x, 0, 1)
    return gaussian_data_x

x_train_gauss = make_gaussian_noise_data(x_train)
x_test_gauss = make_gaussian_noise_data(x_test)

**リスト8.4:ノイズを加えた画像とオリジナル画像との比較**

In [9]:
from IPython.display import display_png


display_png(array_to_img(x_train[0]))
display_png(array_to_img(x_train_gauss[0]))
display_png(array_to_img(x_train_masked[0]))

**リスト8.5:CAEモデルの構築**

In [10]:
autoencoder = Sequential()

# Encoder箇所
autoencoder.add(
    Conv2D(
        16,
        (3, 3),
        1,
        activation='relu', 
        padding='same', 
        input_shape=(28, 28, 1)
    )
)
autoencoder.add(
    MaxPooling2D(
        (2, 2),
        padding='same'
    )
)
autoencoder.add(
    Conv2D(
        8,
        (3, 3),
        1,
        activation='relu', 
        padding='same'
    )
)
autoencoder.add(
    MaxPooling2D(
        (2, 2),
        padding='same'
    )
)

# Decoder箇所
autoencoder.add(
    Conv2D(
        8,
        (3, 3),
        1,
        activation='relu', 
        padding='same'
    )
)
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(
    Conv2D(
        16,
        (3, 3),
        1,
        activation='relu', 
        padding='same'
    )
)
autoencoder.add(UpSampling2D((2, 2)))
autoencoder.add(
    Conv2D(
        1,
        (3, 3),
        1,
        activation='sigmoid', # sigmoid  （0-1なので。ピクセルごとに白/黒の予測）
        padding='same'
    )
)

autoencoder.compile(
    optimizer='adam', 
    loss='binary_crossentropy' # 白(1)/黒(0)の識別問題なのでbinary_crossentropyが使える(?)
) 
initial_weights = autoencoder.get_weights()

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
# binary_crossentropyの実装
# https://github.com/tensorflow/tensorflow/blob/r1.8/tensorflow/python/keras/_impl/keras/losses.py

#def binary_crossentropy(y_true, y_pred):
#  return K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1) # 全ピクセルの平均

In [ ]:
# sigmoidの代わりにlinear +  MSE でも問題なく学習できた
# Relu + MSEも学習できたが生成画像の画質はそれほど良くない感じ

# NGだったパターン
#     activationがsigmoidでロス関数をMSEにするとマスキングノイズの画像の学習が発散してしまった （黒のピクセルが多いから??）
#     activationがReluでロス関数をbinary_crossentropyにするとガウスノイズの画像で発散した。（当たり前か?）

In [ ]:
# 以下のようにMaxPooling2Dを使わずにstride=2のConv2Dを使い、stride=2のConv2DTransposeを使ったモデルでも動作する
# ただし、生成画像の画質はボケた感じ

# 参考:
# https://stackoverflow.com/questions/48226783/what-is-the-the-difference-between-performing-upsampling-together-with-strided-t

autoencoder = Sequential()

# Encoder箇所
autoencoder.add(
    Conv2D(
        16,
        (3, 3),
        2,
        activation='relu', 
        padding='same', 
        input_shape=(28, 28, 1)
    )
)
autoencoder.add(
    Conv2D(
        8,
        (3, 3),
        2,
        activation='relu', 
        padding='same'
    )
)

# Decoder箇所
autoencoder.add(
    Conv2D(
        8,
        (3, 3),
        1,
        activation='relu', 
        padding='same'
    )
)
autoencoder.add(
    Conv2DTranspose(
        16, 
        (3, 3), 
        (2, 2), 
        activation='relu', 
        padding='same'
    )
)
autoencoder.add(
    Conv2DTranspose(
        1, 
        (3, 3), 
        (2, 2), 
        activation='sigmoid', 
        padding='same'
    )
)


**リスト8.6 モデルのサマリを確認**

In [11]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 8)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 16)        1168      
__________

**リスト8.7:ガウシアンノイズを用いて学習**

In [12]:
autoencoder.fit(
                x_train_gauss, # 入力：ガウシアンノイズデータ
                x_train,       # 正解：オリジナル画像
                epochs=10,     # 学習するエポック回数
                batch_size=20, # バッチサイズ
                shuffle=True   # シャッフルデータにする
               )

Epoch 1/10
60000/60000 [==============================]60000/60000 [==============================] - 29s 479us/step - loss: 0.1796

Epoch 2/10
60000/60000 [==============================]60000/60000 [==============================] - 28s 463us/step - loss: 0.1558

Epoch 3/10
60000/60000 [==============================]60000/60000 [==============================] - 28s 464us/step - loss: 0.1511

Epoch 4/10
60000/60000 [==============================]60000/60000 [==============================] - 28s 461us/step - loss: 0.1490

Epoch 5/10
60000/60000 [==============================]60000/60000 [==============================] - 28s 461us/step - loss: 0.1477

Epoch 6/10
60000/60000 [==============================]60000/60000 [==============================] - 28s 461us/step - loss: 0.1468

Epoch 7/10
60000/60000 [==============================]60000/60000 [==============================] - 28s 459us/step - loss: 0.1461

Epoch 8/10
60000/60000 [==============================]60000/60000 [=

**リスト8.8:ガウシアンノイズで学習したモデルでの予測**

In [13]:
gauss_preds = autoencoder.predict(x_test_gauss)

**リスト8.9:ガウシアンノイズ画像、CAEモデル、オリジナル画像の予測の比較**

In [14]:
for i in range(10):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(x_test_gauss[i]))
    display_png(array_to_img(gauss_preds[i]))
    print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


In [16]:
# 試しにノイズなし画像で予測してみると
original_preds = autoencoder.predict(x_test)

for i in range(10):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(original_preds[i]))
    print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


**リスト8.10:CAEモデルの重みを初期化**

In [17]:
autoencoder.set_weights(initial_weights)

**リスト8.11:マスキングノイズを用いて学習**

In [18]:
autoencoder.fit(
                x_train_masked, # 入力：マスキングノイズデータ
                x_train,        # 正解：オリジナル画像
                epochs=10,      # 学習するエポック回数
                batch_size=20,  # バッチサイズ
                shuffle=True    # シャッフルデータにする
               )

Epoch 1/10
60000/60000 [==============================]60000/60000 [==============================] - 27s 455us/step - loss: 0.2050

Epoch 2/10
60000/60000 [==============================]60000/60000 [==============================] - 28s 459us/step - loss: 0.1894

Epoch 3/10
60000/60000 [==============================]60000/60000 [==============================] - 27s 458us/step - loss: 0.1862

Epoch 4/10
60000/60000 [==============================]60000/60000 [==============================] - 27s 455us/step - loss: 0.1845

Epoch 5/10
60000/60000 [==============================]60000/60000 [==============================] - 27s 456us/step - loss: 0.1833

Epoch 6/10
60000/60000 [==============================]60000/60000 [==============================] - 27s 454us/step - loss: 0.1825

Epoch 7/10
60000/60000 [==============================]60000/60000 [==============================] - 27s 455us/step - loss: 0.1819

Epoch 8/10
60000/60000 [==============================]60000/60000 [=

**リスト8.12:マスキングノイズデータで学習したモデルでの予測**

In [19]:
masked_preds = autoencoder.predict(x_test_masked)

**リスト8.13:マスキングノイズ画像、CAEモデル、オリジナル画像の予測の比較**

In [20]:
for i in range(10):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(x_test_masked[i]))
    display_png(array_to_img(masked_preds[i]))
    print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


In [21]:
# 試しにノイズなし画像で予測してみると
original_preds = autoencoder.predict(x_test)

for i in range(10):
    display_png(array_to_img(x_test[i]))
    display_png(array_to_img(original_preds[i]))
    print('-'*25)

-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------


-------------------------
